1. Dataset overview_ Amazon Fine Food reviews(EDA)-

About Dataset
Context
This dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.

Contents
Reviews.csv: Pulled from the corresponding SQLite table named Reviews in database.sqlite
database.sqlite: Contains the table 'Reviews'

Data includes:

Reviews from Oct 1999 - Oct 2012
568,454 reviews
256,059 users
74,258 products
260 users with > 50 reviews

In [54]:
%matplotlib inline

import sqlite3 
import pandas as pd
import numbers as np
import nltk 
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

In [55]:
#connect sql file
# using the SQLite Table to read data.
con = sqlite3.connect("G:/STUDING/[APPLIED]/[AppliedAI] Machine Learning Online Course/Module 3_ Foundations of Natural Language Processing and Machine Learning/1. Real world problem_ Predict rating given product reviews on Amazon/DATA/database.sqlite")

In [56]:
#filtering only positive and negative reviews i.e.
# Not taking into consideration those reviews with score = 3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con)

In [57]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a

def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [58]:
#changing reviews with score less than 3 to be positive and vice—versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative   

In [59]:
filtered_data.shape#*looking at the number of attributes and size of the data
filtered_data.head

<bound method NDFrame.head of             Id   ProductId          UserId                      ProfileName  \
0            1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1            2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2            3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3            4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4            5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...        ...         ...             ...                              ...   
525809  568450  B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
525810  568451  B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
525811  568452  B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
525812  568453  B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
525813  568454  B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

        HelpfulnessNu

2. Data Cleaning_ Deduplication-

It is observed (as shown in the table below) that the reviews data had many duplicate entries.
Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.Following is an example:

In [60]:
display=  pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER By ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


Deduping data

As can be seen above the same user has multiple reviews of the with the same values for
HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing
analysis it was found that
Productld=BOOOHDOPZG Loacker Quadratini Vanilla Wafer Cookies, 8.82-0unce Packages
(Pack of 8)
Productld=BOOOHDL1 RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-0unce Packages
(Pack of 8) and so on

It was inferred after analysis that reviews with same parameters other than Productid belong
the same product just having different flavour or quantity. Hence in order to reduce redundar
was decided to eliminate the rows having same parameters.
The method used for the same was that we first sort the data according to Productld and then just
keep the first similar product rview and delelte the others. for eg. in the above just the review for
ProductId=B000HDLi RQ remains. This method ensures that there is only one representative for
each product and deduplication without sorting would lead to possibility of different representatives
still existing for the same product.  

In [61]:
#Sorting data according to Productid in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)

In [62]:
#deduplication of entry
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace =False,ignore_index=False)
final.shape

(364173, 10)

In [63]:
#check to see how much % od data still remains
(final['Id'].size*1.0/(filtered_data['Id'].size*1.0)*100)


69.25890143662969

Observation:- It was also seen that in two rows given below the value of HelpfulnessNumerator is
greater than HelpfulnessDenominator which is not practically possible hence these two rows too are
removed from calcualtions

In [64]:
display=pd.read_sql_query(""" 
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [65]:
finaly=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [66]:
#Before Starting the next phase of processing lets see the numbers of entry
print(final.shape)

#How man positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364173, 10)


positive    307063
negative     57110
Name: Score, dtype: int64

5. Stop-word removal, Tokenization, Lemmatization (Featurizations - convert text to numeric vectors)-

In [4]:
#text pre-processing
import re 
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleanr= re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanounc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

print(stop)

{'why', 'yourself', 'should', 'wouldn', 'above', 'whom', 'mightn', 'that', "you'd", 'he', 'hers', 'are', 'itself', 'when', "hasn't", 'my', 'been', 'ma', "couldn't", 'aren', 'as', 'too', 'haven', 'wasn', 'herself', 'some', 'will', 'further', "won't", "that'll", 'an', 'than', 'ourselves', 'then', 'his', 'such', 'nor', "aren't", 'so', 'once', 'hadn', 'don', 'not', "mightn't", 'most', 'him', "you've", 'on', "don't", "you'll", "needn't", 'here', 'between', 'needn', 'any', 'or', 'more', 'at', 'if', 'these', "wouldn't", 'while', 'ain', 'just', 'into', 'this', 'now', 'mustn', "didn't", 'off', 'below', 'have', "hadn't", 'a', 'our', 'can', "wasn't", 'after', 'o', 'hasn', 'i', 'you', 'didn', 'because', 'under', 'during', 'isn', 'shan', 'm', 'am', 'down', 'all', 'theirs', 'where', "she's", 'll', "shouldn't", 'there', 'same', 'themselves', 'be', 'being', 'she', 'before', 'each', 'only', 'those', 'couldn', "weren't", 'does', 'with', "haven't", 'weren', 'by', 'me', 'having', 's', 'did', 'had', 'who',